In [242]:
# Import libraries
#import pgmpy.models
#import pgmpy.inference
import numpy as np
import pysmile
import pysmile_license
import os
import pandas as pd
import math
from enum import Enum
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

In [243]:
numSlices = 24

class NodeNames(Enum):
    MITM = "MITM"
    SRM = "SRM"
    UC = "UC"
    UPS = "UPS"
    IMD = "IMD"
    MC = "MC"
    CC = "CC"

PySmile Network

In [244]:
DEBUG = False

#evidenceNodes = [NodeNames.IMD.value, NodeNames.MC.value, NodeNames.CC.value]
evidenceNodes = []

trainFileName = "outTrain4.csv"
testFileName = "outTest4.csv"
networkFileName = "DBN-MITM.xdsl"

classNodesHandles = dict()

In [245]:
def print_node_info(net, node_handle):
    print("Node id/name: " + net.get_node_id(node_handle) + "/" +
    net.get_node_name(node_handle))
    print(" Outcomes: " + " ".join(net.get_outcome_ids(node_handle)))
    parent_ids = net.get_parent_ids(node_handle)
    if len(parent_ids) > 0:
        print(" Parents: " + " ".join(parent_ids))
    child_ids = net.get_child_ids(node_handle)
    if len(child_ids) > 0:
        print(" Children: " + " ".join(child_ids))
    print_cpt_matrix(net, node_handle)
    
def print_cpt_matrix(net, node_handle):
    cpt = net.get_node_definition(node_handle)
    parents = net.get_parents(node_handle)
    dim_count = 1 + len(parents)
    dim_sizes = [0] * dim_count
    for i in range(0, dim_count - 1):
        dim_sizes[i] = net.get_outcome_count(parents[i])
    dim_sizes[len(dim_sizes) - 1] = net.get_outcome_count(node_handle)
    coords = [0] * dim_count
    for elem_idx in range(0, len(cpt)):
        index_to_coords(elem_idx, dim_sizes, coords)
        outcome = net.get_outcome_id(node_handle, coords[dim_count - 1])
        out_str = " P(" + outcome
        if dim_count > 1:
            out_str += " | "
            for parent_idx in range(0, len(parents)):
                if parent_idx > 0:
                    out_str += ","
                parent_handle = parents[parent_idx]
                out_str += net.get_node_id(parent_handle) + "=" + \
                net.get_outcome_id(parent_handle, coords[parent_idx])
        prob = cpt[elem_idx]
        out_str += ")=" + str(prob)
        print(out_str)
    
        
def index_to_coords(index, dim_sizes, coords):
    prod = 1
    for i in range(len(dim_sizes) - 1, -1, -1):
        coords[i] = int(index / prod) % dim_sizes[i]
        prod *= dim_sizes[i]

def pint_time_cpt_marix(net, nodeHandle):
    timeCPT = net.get_node_temporal_definition(nodeHandle, 1)
    
        
def plot_time_CPT(net, nodeHandle):    
    cpt = net.get_node_temporal_definition(nodeHandle, 1)
    print(len(cpt))
    print("###")
    
def print_net_info(net, unrolled = True):
    for n in net.get_all_nodes():
        print_node_info(net, n)
        if not unrolled and net.get_node_id(n) == NodeNames.UPS.value:
            plot_time_CPT(net, n)
            
def calc_stat(confMatrix, outcome, type='P'):
    TP = confMatrix[outcome][outcome]
    FP = 0
    TN = 0
    FN = 0
    for i in range(0, len(confMatrix)):
        for j in range(0, len(confMatrix[i])):
            if i == outcome and j != outcome:
                FP += confMatrix[i][j]
            if j == outcome and i != outcome:
                FN += confMatrix[i][j]
            if i != outcome and j != outcome:
                TN += confMatrix[i][j] 
                
    if type == 'P':
        if TP + FP == 0: return float('nan')
        return TP / (TP + FP)
    if type == 'A':
        if TP + TN + FP + FN == 0: return float('nan')
        return  (TP+TN) / (TP + TN + FP + FN)
    if type == 'R':
        if TP + FN == 0: return float('nan')
        return TP / (TP + FN)
    if type == 'F':
        if (2*TP)+FP+FN == 0: return float('nan')
        return (2*TP)/((2*TP)+FP+FN)        

def print_validator_results(net, originalSliceCount, validator, nodeId): 
    nodeHandle = classNodesHandles[nodeId]
    outcomeCount = net.get_outcome_count(nodeHandle)
    accMtrx = np.zeros((outcomeCount, 2, originalSliceCount))
    precMtrx = np.zeros((outcomeCount, 2, originalSliceCount))
    recMtrx = np.zeros((outcomeCount, 2, originalSliceCount))
    fMtrx = np.zeros((outcomeCount, 2, originalSliceCount))
    
    
    for slice in range(1, originalSliceCount):
        if DEBUG: print("### Slice " + str(slice) + " ###")
        nodeHandle = classNodesHandles[nodeId + "_" + str(slice)]
        cm = validator.get_confusion_matrix(nodeHandle)
        for i in range(0, outcomeCount):
            acc = calc_stat(cm, i, 'A')
            # If the calculated statistic is NaN assign 0 (see next line)
            accMtrx[i][0][slice] = acc if not math.isnan(acc) else 0
            # If the calculated statistic is NaN assign weight 0 to it
            # otherwise use the number of elements belonging to that class as weight
            accMtrx[i][1][slice] = np.sum(cm[:][i]) if not math.isnan(acc) else 0
            prec = calc_stat(cm, i, 'P')
            precMtrx[i][0][slice] = prec if not math.isnan(prec) else 0
            precMtrx[i][1][slice] = np.sum(cm[:][i]) if not math.isnan(prec) else 0
            rec = calc_stat(cm, i, 'R')
            recMtrx[i][0][slice] = rec if not math.isnan(rec) else 0
            recMtrx[i][1][slice] = np.sum(cm[:][i]) if not math.isnan(rec) else 0
            f = calc_stat(cm, i, 'F')
            fMtrx[i][0][slice] = f if not math.isnan(f) else 0
            fMtrx[i][1][slice] = np.sum(cm[:][i]) if not math.isnan(f) else 0
            
            if DEBUG:
                print("Accuracy for " + nodeId + str(i) + ": " + str(acc))
                print("Precision for " + nodeId + str(i) + ": " + str(prec))
                print("Recall for " + nodeId + str(i) + ": " + str(rec))    
        if DEBUG:    
            print("** Confusion Matrix **")
            for i in range(0, outcomeCount):
                print(cm[i])
            print("")
    
    # Calculates weighted averages (each statistic of each timeslice is weighted
    # by the number of elements nelonging to that class in that timeslice)
    avgAccOut = np.zeros(outcomeCount)
    avgPrecOut = np.zeros(outcomeCount)
    avgRecOut = np.zeros(outcomeCount)
    avgFOut = np.zeros(outcomeCount)
    for i in range(0, outcomeCount):
        avgAccOut[i] = np.sum(accMtrx[i][0]*accMtrx[i][1])/np.sum(accMtrx[i][1])
        avgPrecOut[i] = np.sum(precMtrx[i][0]*precMtrx[i][1])/np.sum(precMtrx[i][1])
        avgRecOut[i] = np.sum(recMtrx[i][0]*recMtrx[i][1])/np.sum(recMtrx[i][1])
        avgFOut[i] = np.sum(fMtrx[i][0]*fMtrx[i][1])/np.sum(fMtrx[i][1])
        
        
        print("Average Accuracy for " + nodeId + str(i) + ": " + str(avgAccOut[i]))
        print("Average Precision for " + nodeId + str(i) + ": " + str(avgPrecOut[i]))
        print("Average Recall for " + nodeId + str(i) + ": " + str(avgRecOut[i]))
        print("Average F-score for " + nodeId + str(i) + ": " + str(avgFOut[i]))
        print("") 
    
    out = {}
    out['A'] = np.average(avgAccOut)
    out['P'] = np.average(avgPrecOut) 
    out['R'] = np.average(avgRecOut) 
    out['F'] = np.average(avgFOut)
    return out
        
            
def eraseDefinitions(net):
    nodes = net.get_all_node_ids()
    for node in nodes:
        cpt = net.get_node_definition(node)
        numOutcomes = net.get_outcome_count(node)
        p = 1 / numOutcomes
        for i in range(0, len(cpt)):
            cpt[i] = p
        net.set_node_definition(node, cpt)

def eraseTemporalDefinitions(net):
    nodes = net.get_all_node_ids()
    for node in nodes:
        try:
            cpt = net.get_node_temporal_definition(node, 1)
            numOutcomes = net.get_outcome_count(node)
            p = 1 / numOutcomes
            for i in range(0, len(cpt)):
                cpt[i] = p
            net.set_node_temporal_definition(node, 1, cpt)  
        except:
            pass

def partializeEvidence(df, evidenceNodes, sliceCount):
    for evidenceNode in evidenceNodes:
        for slice in range(1, sliceCount):
            if 1 <= slice <= 20:
                colName = evidenceNode + "_" + str(slice)
                df = df.drop(colName, axis=1)
    return df

In [246]:
# Create and read the DBN from file
net = pysmile.Network()
ds = pysmile.learning.DataSet()

net.read_file(os.getcwd() + "/../../../Genie-DBN/" + networkFileName)
    

Erase CPTs before training

In [247]:
eraseDefinitions(net)
eraseTemporalDefinitions(net)

Traininig

In [248]:
df = pd.read_csv(os.getcwd() + "/" + trainFileName)
ds.read_pandas_dataframe(df)

matching = ds.match_network(net)
em = pysmile.learning.EM()
# Small data variatons correspond to big changes
em.set_relevance(0)
res = em.learn(ds, net, matching)

print("N-LL: " + str(em.get_last_score()))

N-LL: -183900.29110254004


Test performances

In [249]:
def testPerf(net, nodeName, testDs = None):
    if testDs is None:
        testDs = pd.read_csv(os.getcwd() + "/" + testFileName)
        
    ds.read_pandas_dataframe(testDs)
    
    unrolledNet = net.unroll().unrolled     
    matching = ds.match_network(unrolledNet)
    validator = pysmile.learning.Validator(unrolledNet, ds, matching)
    # Set class nodes (those that will not be considered as evidence nodes)
    for elem in NodeNames.__members__:
        if elem not in evidenceNodes:
            classNodesHandles[elem] = unrolledNet.get_node(elem)
            validator.add_class_node(classNodesHandles[elem])
            for slice in range(1, net.get_slice_count()):
                elemCat = elem + "_" + str(slice)
                classNodesHandles[elemCat] = unrolledNet.get_node(elemCat)
                validator.add_class_node(classNodesHandles[elemCat])
    # Test the predctions on the class nodes            
    validator.test()
    #print(validator.get_result_dataset())
    if DEBUG:
        print_net_info(unrolledNet, unrolled=True)
    
    return print_validator_results(unrolledNet, net.get_slice_count(), validator, nodeName)



In [250]:
def compare_files(trainFileName, testFileName):
    dfTrain = pd.read_csv(os.getcwd() + "/" + trainFileName)
    dfTest = pd.read_csv(os.getcwd() + "/" + testFileName)
    
    dfOut = pd.concat([dfTrain, dfTest]).drop_duplicates(keep="first")
    dups  = dfTrain.shape[0] + dfTest.shape[0] - dfOut.shape[0]
    print(dfTrain.shape[0])
    print(dfTest.shape[0])
    print("Duplicate rows: " + str(dups))
    
compare_files("outTrain4.csv", "outTrain4.csv")

4000
4000
Duplicate rows: 4003


Test learning capabilities

In [251]:
popSize = 4000
nlls = []
accs = []
precs = []
recs = []
fs = []
df = pd.read_csv(os.getcwd() + "/" + trainFileName)
kfold = KFold(n_splits=10)
for trainDs, testDs in kfold.split(df[0:popSize]):
    ds.read_pandas_dataframe(df.iloc[trainDs])
    matching = ds.match_network(net)
    em = pysmile.learning.EM()
    em.set_randomize_parameters(True)
    em.set_seed(10)
    #eraseDefinitions(net)
    #eraseTemporalDefinitions(net)
    res = em.learn(ds, net, matching)
    nlls.append(em.get_last_score())
    print("N-LL: " + str(em.get_last_score()))
    
    nodeName = NodeNames.UPS.value
    out = testPerf(net, nodeName, testDs = partializeEvidence(df.iloc[testDs], evidenceNodes, net.get_slice_count()))
    accs.append(out['A'])
    precs.append(out['P'])
    recs.append(out['R'])
    fs.append(out['F'])

print("Average Accuracy on node " + nodeName + ": " + str(np.average(accs)))
print("Average Precision on node " + nodeName + ": " + str(np.average(precs)))
print("Average Recall on node " + nodeName + ": " + str(np.average(recs)))
print("Average F-score on node " + nodeName + ": " + str(np.average(fs)))


N-LL: -165861.66481208976
Average Accuracy for UPS0: 0.8723580941810687
Average Precision for UPS0: 0.8558744252417948
Average Recall for UPS0: 0.9151528343831048
Average F-score for UPS0: 0.8150308128760482

Average Accuracy for UPS1: 0.7856172183419375
Average Precision for UPS1: 0.8171879744913453
Average Recall for UPS1: 0.7911677814938686
Average F-score for UPS1: 0.7137140566674347

N-LL: -165871.51980629924
Average Accuracy for UPS0: 0.8610503273945485
Average Precision for UPS0: 0.8317344297243795
Average Recall for UPS0: 0.9219434273160014
Average F-score for UPS0: 0.7957411892921568

Average Accuracy for UPS1: 0.7591435872073855
Average Precision for UPS1: 0.8226178700956149
Average Recall for UPS1: 0.7519589178356715
Average F-score for UPS1: 0.6947314258056108

N-LL: -165647.2211469137
Average Accuracy for UPS0: 0.8573521820741437
Average Precision for UPS0: 0.8390427029563585
Average Recall for UPS0: 0.9075307606263981
Average F-score for UPS0: 0.7957817362753755

Average 

# 4 states for SRM ecc... (complete evidence)
Average Accuracy on node UPS: 0.7691188647062448
Average Precision on node UPS: 0.7580515072136919
Average Recall on node UPS: 0.7915767405552715
Average F-score on node UPS: 0.7707809782488163

# 4 states for SRM ecc... (evidence just on odd timeslices)
Average Accuracy on node UPS: 0.7677149087383711
Average Precision on node UPS: 0.7589832019963916
Average Recall on node UPS: 0.7915573192540584
Average F-score on node UPS: 0.7708178971171644

# 2 states for all the nodes (complete evidence)
Average Accuracy on node UPS: 0.8066597430345286
Average Precision on node UPS: 0.8114346737200531
Average Recall on node UPS: 0.7885880802092043
Average F-score on node UPS: 0.790012708581611

# 2 states for all the nodes (evidence just on odd timeslices)
Average Accuracy on node UPS: 0.8116308516361525
Average Precision on node UPS: 0.8152992459442799
Average Recall on node UPS: 0.7921912497608095
Average F-score on node UPS: 0.7927440542420979